In [1]:
# Importar librerías
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month
import os

In [3]:
spark = SparkSession.builder \
    .appName("SECOP_Ingesta") \
    .master("local[*]") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [4]:
print(f"Spark Version: {spark.version}")
print(f"Spark Master: {spark.sparkContext.master}")

Spark Version: 3.5.0
Spark Master: local[*]


In [5]:
import requests
import json


In [6]:
api_url = "https://www.datos.gov.co/resource/jbjy-vk9h.json?$query=SELECT%0A%20%20%60nombre_entidad%60%2C%0A%20%20%60nit_entidad%60%2C%0A%20%20%60departamento%60%2C%0A%20%20%60ciudad%60%2C%0A%20%20%60localizaci_n%60%2C%0A%20%20%60orden%60%2C%0A%20%20%60sector%60%2C%0A%20%20%60rama%60%2C%0A%20%20%60entidad_centralizada%60%2C%0A%20%20%60proceso_de_compra%60%2C%0A%20%20%60id_contrato%60%2C%0A%20%20%60referencia_del_contrato%60%2C%0A%20%20%60estado_contrato%60%2C%0A%20%20%60codigo_de_categoria_principal%60%2C%0A%20%20%60descripcion_del_proceso%60%2C%0A%20%20%60tipo_de_contrato%60%2C%0A%20%20%60modalidad_de_contratacion%60%2C%0A%20%20%60justificacion_modalidad_de%60%2C%0A%20%20%60fecha_de_firma%60%2C%0A%20%20%60fecha_de_inicio_del_contrato%60%2C%0A%20%20%60fecha_de_fin_del_contrato%60%2C%0A%20%20%60condiciones_de_entrega%60%2C%0A%20%20%60tipodocproveedor%60%2C%0A%20%20%60documento_proveedor%60%2C%0A%20%20%60proveedor_adjudicado%60%2C%0A%20%20%60es_grupo%60%2C%0A%20%20%60es_pyme%60%2C%0A%20%20%60habilita_pago_adelantado%60%2C%0A%20%20%60liquidaci_n%60%2C%0A%20%20%60obligaci_n_ambiental%60%2C%0A%20%20%60obligaciones_postconsumo%60%2C%0A%20%20%60reversion%60%2C%0A%20%20%60origen_de_los_recursos%60%2C%0A%20%20%60destino_gasto%60%2C%0A%20%20%60valor_del_contrato%60%2C%0A%20%20%60valor_de_pago_adelantado%60%2C%0A%20%20%60valor_facturado%60%2C%0A%20%20%60valor_pendiente_de_pago%60%2C%0A%20%20%60valor_pagado%60%2C%0A%20%20%60valor_amortizado%60%2C%0A%20%20%60valor_pendiente_de%60%2C%0A%20%20%60valor_pendiente_de_ejecucion%60%2C%0A%20%20%60estado_bpin%60%2C%0A%20%20%60c_digo_bpin%60%2C%0A%20%20%60anno_bpin%60%2C%0A%20%20%60saldo_cdp%60%2C%0A%20%20%60saldo_vigencia%60%2C%0A%20%20%60espostconflicto%60%2C%0A%20%20%60dias_adicionados%60%2C%0A%20%20%60puntos_del_acuerdo%60%2C%0A%20%20%60pilares_del_acuerdo%60%2C%0A%20%20%60urlproceso%60%2C%0A%20%20%60nombre_representante_legal%60%2C%0A%20%20%60nacionalidad_representante_legal%60%2C%0A%20%20%60domicilio_representante_legal%60%2C%0A%20%20%60tipo_de_identificaci_n_representante_legal%60%2C%0A%20%20%60identificaci_n_representante_legal%60%2C%0A%20%20%60g_nero_representante_legal%60%2C%0A%20%20%60presupuesto_general_de_la_nacion_pgn%60%2C%0A%20%20%60sistema_general_de_participaciones%60%2C%0A%20%20%60sistema_general_de_regal_as%60%2C%0A%20%20%60recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_%60%2C%0A%20%20%60recursos_de_credito%60%2C%0A%20%20%60recursos_propios%60%2C%0A%20%20%60ultima_actualizacion%60%2C%0A%20%20%60codigo_entidad%60%2C%0A%20%20%60codigo_proveedor%60%2C%0A%20%20%60fecha_inicio_liquidacion%60%2C%0A%20%20%60fecha_fin_liquidacion%60%2C%0A%20%20%60objeto_del_contrato%60%2C%0A%20%20%60duraci_n_del_contrato%60%2C%0A%20%20%60nombre_del_banco%60%2C%0A%20%20%60tipo_de_cuenta%60%2C%0A%20%20%60n_mero_de_cuenta%60%2C%0A%20%20%60el_contrato_puede_ser_prorrogado%60%2C%0A%20%20%60fecha_de_notificaci_n_de_prorrogaci_n%60%2C%0A%20%20%60nombre_ordenador_del_gasto%60%2C%0A%20%20%60tipo_de_documento_ordenador_del_gasto%60%2C%0A%20%20%60n_mero_de_documento_ordenador_del_gasto%60%2C%0A%20%20%60nombre_supervisor%60%2C%0A%20%20%60tipo_de_documento_supervisor%60%2C%0A%20%20%60n_mero_de_documento_supervisor%60%2C%0A%20%20%60nombre_ordenador_de_pago%60%2C%0A%20%20%60tipo_de_documento_ordenador_de_pago%60%2C%0A%20%20%60n_mero_de_documento_ordenador_de_pago%60%2C%0A%20%20%60documentos_tipo%60%2C%0A%20%20%60descripcion_documentos_tipo%60%0AWHERE%0A%20%20caseless_one_of(%60departamento%60%2C%20%22Distrito%20Capital%20de%20Bogot%C3%A1%22)%0A%20%20AND%20%60fecha_de_firma%60%0A%20%20%20%20%20%20%20%20BETWEEN%20%222025-10-01T19%3A14%3A39%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20%20%20%20%20AND%20%222025-12-31T23%3A59%3A00%22%20%3A%3A%20floating_timestamp"

26/01/31 21:25:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
response = requests.get(api_url)
data = response.json()

In [9]:
pip install sodapy


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from sodapy import Socrata

client = Socrata("www.datos.gov.co", None)

results = client.get(
    "jbjy-vk9h",
    query="""
    SELECT *
    WHERE 
        departamento = "Distrito Capital de Bogotá"
    AND
        fecha_de_firma >= '2025-01-01T00:00:00'
    AND
        fecha_de_firma < '2025-04-01T00:00:00'
    LIMIT 150000
    """
)


In [11]:
len(results)

132642

In [12]:

import pandas as pd
import os

DATA_PATH = "/app/data"

df = pd.DataFrame.from_records(results)

df.to_csv(
    os.path.join(DATA_PATH, "SECOP_II_Contratos_Electronicos_Q1_2025.csv"),
    index=False,
    encoding="utf-8"
)

print("CSV guardado correctamente en /app/data")



CSV guardado correctamente en /app/data


In [13]:
df_spark = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/app/data/SECOP_II_Contratos_Electronicos_Q1_2025.csv")
)


In [14]:
len(df_spark.columns)


87

In [15]:
df_spark.printSchema()


root
 |-- nombre_entidad: string (nullable = true)
 |-- nit_entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- localizaci_n: string (nullable = true)
 |-- orden: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- rama: string (nullable = true)
 |-- entidad_centralizada: string (nullable = true)
 |-- proceso_de_compra: string (nullable = true)
 |-- id_contrato: string (nullable = true)
 |-- referencia_del_contrato: string (nullable = true)
 |-- estado_contrato: string (nullable = true)
 |-- codigo_de_categoria_principal: string (nullable = true)
 |-- descripcion_del_proceso: string (nullable = true)
 |-- tipo_de_contrato: string (nullable = true)
 |-- modalidad_de_contratacion: string (nullable = true)
 |-- justificacion_modalidad_de: string (nullable = true)
 |-- fecha_de_firma: string (nullable = true)
 |-- fecha_de_inicio_del_contrato: string (nullable = true)
 |-- fecha_de_fin_del_contrato: string

In [16]:
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql.types import DoubleType, LongType, IntegerType


In [17]:
fecha_cols = [
    "fecha_de_firma",
    "fecha_de_inicio_del_contrato",
    "fecha_de_fin_del_contrato",
    "ultima_actualizacion",
    "fecha_inicio_liquidacion",
    "fecha_fin_liquidacion",
    "fecha_de_notificaci_n_de_prorrogaci_n"
]

for c in fecha_cols:
    df_spark = df_spark.withColumn(
        c,
        to_timestamp(col(c))
    )


In [18]:
double_cols = [
    "valor_del_contrato",
    "valor_de_pago_adelantado",
    "valor_facturado",
    "valor_pendiente_de_pago",
    "valor_pagado",
    "valor_amortizado",
    "valor_pendiente_de",
    "valor_pendiente_de_ejecucion",
    "presupuesto_general_de_la_nacion_pgn",
    "sistema_general_de_participaciones",
    "recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_",
    "recursos_de_credito",
    "recursos_propios"
]

for c in double_cols:
    df_spark = df_spark.withColumn(
        c,
        col(c).cast(DoubleType())
    )


In [19]:
int_cols = [
    "dias_adicionados",
    "codigo_entidad",
    "codigo_proveedor",
    "sistema_general_de_regal_as"
]

for c in int_cols:
    df_spark = df_spark.withColumn(
        c,
        col(c).cast(IntegerType())
    )


In [20]:
long_cols = [
    "saldo_cdp",
    "saldo_vigencia"
]

for c in long_cols:
    df_spark = df_spark.withColumn(
        c,
        col(c).cast(LongType())
    )


In [21]:
df_spark.printSchema()


root
 |-- nombre_entidad: string (nullable = true)
 |-- nit_entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- localizaci_n: string (nullable = true)
 |-- orden: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- rama: string (nullable = true)
 |-- entidad_centralizada: string (nullable = true)
 |-- proceso_de_compra: string (nullable = true)
 |-- id_contrato: string (nullable = true)
 |-- referencia_del_contrato: string (nullable = true)
 |-- estado_contrato: string (nullable = true)
 |-- codigo_de_categoria_principal: string (nullable = true)
 |-- descripcion_del_proceso: string (nullable = true)
 |-- tipo_de_contrato: string (nullable = true)
 |-- modalidad_de_contratacion: string (nullable = true)
 |-- justificacion_modalidad_de: string (nullable = true)
 |-- fecha_de_firma: timestamp (nullable = true)
 |-- fecha_de_inicio_del_contrato: timestamp (nullable = true)
 |-- fecha_de_fin_del_contrato: 

In [30]:
import os

os.getcwd()


'/app/notebooks'

In [31]:
os.listdir("/")


['run',
 'lib64',
 'usr',
 'home',
 'tmp',
 'media',
 'dev',
 'boot',
 'proc',
 'etc',
 'srv',
 'var',
 'bin',
 'sys',
 'sbin',
 'mnt',
 'root',
 'opt',
 'lib',
 'app',
 '.dockerenv']

In [32]:
import os

os.listdir("/app")


['notebooks', 'data']

In [33]:
import os

os.listdir("/app/data")


['.ipynb_checkpoints',
 'lakehouse',
 'SECOP_II_Contratos_Electronicos.csv',
 'SECOP_II_Contratos_Electronicos_Q1_2025.csv']

In [34]:
os.listdir("/app/data/lakehouse")


['bronze']

In [36]:
import os
os.listdir("/app/data/lakehouse/bronze/secop")


['.part-00000-0a648699-90de-4089-91f9-ca4cd50ad393-c000.snappy.parquet.crc',
 '.part-00000-5be63bcb-f9f0-4b4a-838e-5363fbe3f083-c000.snappy.parquet.crc',
 '.part-00000-75ce9916-16da-436d-a94b-d16f183ae5bb-c000.snappy.parquet.crc',
 '.part-00000-b4643773-c2ff-4e68-bcd5-dfcd8bca9dbe-c000.snappy.parquet.crc',
 '.part-00001-b3c9f5ef-31af-48b7-b66e-d5253e4795f4-c000.snappy.parquet.crc',
 'part-00000-0a648699-90de-4089-91f9-ca4cd50ad393-c000.snappy.parquet',
 'part-00000-5be63bcb-f9f0-4b4a-838e-5363fbe3f083-c000.snappy.parquet',
 'part-00000-75ce9916-16da-436d-a94b-d16f183ae5bb-c000.snappy.parquet',
 'part-00000-b4643773-c2ff-4e68-bcd5-dfcd8bca9dbe-c000.snappy.parquet',
 'part-00001-b3c9f5ef-31af-48b7-b66e-d5253e4795f4-c000.snappy.parquet',
 '_delta_log']